In [1]:
# !pip install datasets transformers[torch]

In [33]:
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, BertModel, Trainer, TrainingArguments

X, y의 input type
```python
sklearn.model -> array
torch.nn.model -> tensor
transformers -> Dataset

In [3]:
# 텍스트 정규화 함수
# (text:str)->str: 부분은 다른 사람과 함께 코드를 사용할 때 일종의 힌트를 주는 것
# normalize_token_text() 까지 입력하면 뜸
# text:str = "text라는 매개변수의 type은 문자형을 권장한다"
# -> str: = "출력 결과의 type이 문자형이다"
def normalize_token_text(text : str) -> str:
    text = re.sub(r"[^가-힣0-9a-zA-z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [4]:
# 데이터 로드
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
# document의 결측치 제거
df.dropna(subset=['document'], inplace=True)

# 텍스트 정규화
df['document'] = df['document'].map(normalize_token_text)

# document 컬럼에서 길이가 1 이하인 행 제거
df = df.loc[ df['document'].str.len() > 1 ]

# 중복 데이터 제거
df.drop_duplicates(subset=['document'], inplace=True)

# 라벨 컬럼의 데이터를 숫자형으로 변환
df['label'] = df['label'].astype('int64')

# 데이터프레임의 인덱스를 초기화하고 기존 인덱스 제거
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144775 entries, 0 to 144774
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        144775 non-null  int64 
 1   document  144775 non-null  object
 2   label     144775 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.3+ MB


In [6]:
# 테스트를 위해 일정 구간만 선택
df2 = df.head(1000)

In [7]:
# DataFrame을 train, text 셋으로 분할
train_df, test_df = train_test_split(
    df2, test_size=0.2, random_state=42, stratify=df2['label']
)

# BERT 모델에서 사용하는 데이터 타입으로 변경
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))

In [8]:
train_ds
# pandas와 크게 차이 없는 형태

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 800
})

In [9]:
# 토큰화
# AutoTokenizer를 이용하여 BERT 모델에서 사용하는 토큰화 작업을 로드
# 모델 이름 지정 - skt에서 만든 한글 기반 BERT 모델
MODEL_NAME = "skt/kobert-base-v1"

# use_fast 매개변수
#  False: 기본 토크나이저(python 기반) 사용
    # KoBERT 모델은 sentenpiece 기반 토큰화 -> 기본 모델 사용을 권장
#  True: 빠른 토크나이저(Rust, C 기반) 사용
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

In [10]:
def tok_fn(batch):
    # batch: 데이터의 묶음
    # truncation: 문장이 최대 입력 길이를 초과했을 때, 자동으로 자를 것인가?
    # max_length: 최대 입력 길이
    result = tokenizer( batch['document'], truncation=True, max_length=128 )
    # result -> input_ids, attention_mask, token_type_ids 등을 포함
    return result

In [11]:
# Dataset 타입의 데이터에서 데이터들의 묶음(batch)을 tok_fn에 대입하여 새로운 Dataset 생성
train_tok = train_ds.map(tok_fn, batched=True, remove_columns=['id', 'document'])
test_tok = test_ds.map(tok_fn, batched=True, remove_columns=['id', 'document'])

Map: 100%|██████████| 200/200 [00:00<00:00, 4178.51 examples/s]


In [12]:
train_tok
# input_ids: 문장 토큰의 숫자 인덱스 (단어 사전에서 해당 단어의 위치)
#   --> 단어 사전의 초기 인덱스들은 특수 토큰
# token_type_ids: 문장 구분용 인덱스 (0: 첫 번재 문장, 1: 두 번째 문장, 2: 버그)
# attention_mask: 패딩 토큰과 실제 토큰 분류 (0: 패딩 토큰, 1: 실제 토큰)

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 800
})

In [13]:
train_tok['input_ids']

Column([[4368, 3245, 628, 6416, 7096, 4303, 5348, 5760, 7318, 2044, 6116, 7227, 5860, 2081, 7138, 7850, 6366, 4102, 1458, 3978, 5400, 517, 6389, 55, 1458, 3093, 6067, 7850, 3394, 4841, 4102, 4468, 1410, 6389, 54, 54, 4368, 1435, 3647, 1397, 6896, 3194, 5770, 6116, 2355, 6197, 3647, 2265, 7295, 6896, 3673, 3524, 7292, 7096, 6844, 54, 54, 517, 6896, 6225, 6797, 3, 2], [2870, 6116, 7848, 6003, 2870, 3, 2], [1434, 3774, 4524, 2149, 5920, 3, 2], [1185, 6150, 6456, 6456, 5868, 6122, 5439, 2258, 7318, 5400, 7811, 6812, 5886, 5655, 6999, 2355, 6060, 5782, 6806, 5859, 7350, 7350, 5644, 6855, 7361, 7316, 5655, 54, 6812, 6389, 3301, 6553, 5980, 6496, 7609, 7245, 1861, 7350, 7350, 5644, 6198, 7347, 7361, 7316, 5664, 5943, 2563, 6461, 6122, 6301, 7899, 7350, 5384, 6301, 7899, 7331, 6286, 7207, 7096, 7141, 5929, 3931, 5778, 3647, 5922, 6499, 7788, 0, 7142, 5850, 1460, 5644, 6855, 7361, 7316, 5655, 6999, 3, 2], [3498, 6928, 7996, 517, 442, 453, 6116, 1733, 6371, 5330, 2593, 7920, 54, 4525, 533, 5713,

In [14]:
train_tok['token_type_ids']
# 'token_type_ids' 키 값이 없어도 사용 가능
# 버그(2)가 있음에도 사용 가능한 이유는?

Column([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]])

#### cf. `token_type_ids`의 '2'와 학습 문제

1. `token_type_ids`의 원래 역할 (0 또는 1)

    `token_type_ids` (또는 Segment IDs)는 BERT 모델의 원본 설계에서 **두 개의 문장**(`Sentence A`와 `Sentence B`)을 구별하기 위해 사용됩니다.

    * **0**: `Sentence A`의 토큰임을 나타냄.
    * **1**: `Sentence B`의 토큰임을 나타냄.

    이 값은 **NSP (Next Sentence Prediction)** 와 같은 사전 학습(Pre-training) 태스크에서 문장 쌍을 입력할 때 필수적입니다.

2. KoBERT와 단일 문장의 경우

    제시된 데이터는 **단일 리뷰 텍스트**만 포함하고 있습니다. 즉, `Sentence A`와 `Sentence B`를 구분할 필요가 없습니다.

    * **KoBERT의 경우**: KoBERT의 `AutoTokenizer`는 기본적으로 문장 구분이 필요 없는 **단일 문장 입력**에 대해서는 모든 토큰에 **`0`** (첫 번째 세그먼트)을 할당합니다.
    * **패딩(Padding) 및 특수 토큰**: 시퀀스의 길이를 맞추기 위해 추가되는 **패딩 토큰**과 문장의 끝을 알리는 특수 토큰(예: `[SEP]`, `[CLS]`)에는 토큰 타입 ID가 할당되는데, 이 과정에서 **`2`라는 값이 들어갔다**는 것은 **토크나이저 내부의 처리 방식** 때문입니다.

3. '2'가 있어도 학습에 문제가 없는 이유 (핵심)

    이 토큰 타입 ID에 `2`라는 값이 포함되어 있더라도, **모델의 최종 예측(감정 분류)**에는 심각한 문제가 발생하지 않습니다.

    * **학습 방식의 변화**: BERT가 **사전 학습(Pre-training)**을 마친 후, 우리가 수행하는 **감정 분석**은 **전이 학습(Fine-tuning)** 단계입니다. 이 단계에서는 모델이 주로 **`input_ids`**와 **`attention_mask`**를 통해 텍스트의 의미적 특징을 파악하는 데 집중합니다.
    * **`token_type_ids`의 중요성 감소**: Fine-tuning 단계, 특히 **단일 문장 분류**에서는 **`token_type_ids`의 역할이 최소화**됩니다. 모델은 이미 사전 학습 단계에서 이 타입 ID의 의미(0은 A, 1은 B)를 학습했지만, 단일 문장 환경에서는 이 정보가 예측 성능에 큰 영향을 미치지 않습니다.
    * **무시 가능한 오류**: 토크나이저의 구현상 `[SEP]`이나 패딩 토큰 이후에 `2`가 할당되었을 수 있지만, 모델은 이 값을 심각한 오류로 간주하지 않고, **단순히 인코딩된 정수** 중 하나로 처리합니다. 모델의 분류 레이어는 입력된 최종 특징 벡터를 기반으로만 작동하기 때문에, 이 미세한 타입 ID의 차이는 예측 결과에 큰 영향을 미치지 않습니다.

**결론**: `token_type_ids`의 `2`는 일반적인 사용법은 아니지만, **단일 문장 Fine-tuning 환경**에서는 모델이 주로 의미 정보(단어 토큰)에 집중하기 때문에 **안전하게 무시**됩니다.

토큰화된 문장들의 인코딩까지 완료함

---

In [15]:
# BERT 분류 모델 정의

class BERTClsHead(nn.Module):
    # 생성자 함수 - 부모 클래스의 초기 매개변수 호출
    def __init__(self, model_name, num_label = 2, dropout = 0.1):
        # model_name: 로드할 모델의 이름
        # num_label: 분류 클래스의 개수
        # dropout: 데이터 소실 비율 (과적합 방지)
        
        # 설정 초기화
        super().__init__()

        # 사전에 학습된 BERT 모델 로드 (백본)
        self.backbone = BertModel.from_pretrained(model_name)
        
        # BERT 모델에서의 output의 차원 수 -> 768
        hidden = self.backbone.config.hidden_size

        # 과적합 방지를 위한 dropout
        self.dropout = nn.Dropout(dropout)
        
        # 선형 모델 (2개의 class를 '분류'하는 모델)
        # num_label이 2라면 2개, 3이라면 3 이상으로 분류
        self.classifier = nn.Linear(hidden, num_label)

        # 안정성을 위해 확인차, 패딩 토큰의 아이디 값을 백본 설정에 패딩 아이디에 대입
        self.backbone.config.pad_token_id = tokenizer.pad_token_id

        # 초기 설정 완료
    
    # 순전파 함수 생성
    def forward(self, input_ids = None, attention_mask = None, labels = None, **kwargs):
        # inpit_ids: 토큰화, 인코딩 처리가 완료된 문장
        # attention_mask: 실제 단어인가? 패딩 단어인가?
        # labels: 학습 시 정답의 라벨 (없으면 추론 모드)
        # **kwargs: 추가하고 싶은 게 있으면 받아줌

        # 백본에 입력 데이터를 대입
        out = self.backbone(input_ids= input_ids , attention_mask= attention_mask)

        # [CLS] 토큰 벡터를 추출
        # 입력된 문장의 첫 번째 토큰은 언제나 [CLS] 이므로, [CLS] 토큰 벡터를 추출한다는 것은 해당 문장 전체를 추출한다는 것을 의미.
            # [ [ '[CLS]' ... ]
            #   [ '[CLS]' ... ]
            #   ...
            # ]
        pooled = out.last_hidden_state[:, 0]

        # 일정 비율의 데이터 소실
        drop_out_data = self.dropout(pooled)

        logits = self.classifier(drop_out_data)

        result = {'Logits': logits}

        # labels에 데이터가 존재한다면 손실 계산
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
            result['loss'] = loss

        return result

In [16]:
# 모델 생성
model = BERTClsHead(MODEL_NAME)

In [17]:
# 평가 함수 정의 (모델의 예측 결과(확률 또는 로짓)를 받아 정확도, F1-Score를 계산하여 반환)
def metrics(eval_pred):
    # eval_pred: 예측값 + 실젯값(0 또는 1)이 합쳐진 데이터
    logits, y = eval_pred   # 예측값, 실젯값으로 나눠줌
    # logits [ x.xxx, x.xxx ]
    pred = logits.argmax(-1)
    return { 
        'accuracy_score': accuracy_score(y, pred),
        'f1_score': f1_score(y, pred)
    }

In [18]:
# TrainingArguments -> Trainer가 학습할 때 사용한 각종 설정 값들을 지정하는 객체
args = TrainingArguments(
    # 학습된 모델의 결과들을 저장할 디렉토리 지정
    output_dir= "./kobert_from_bertmodel",
    # 배치의 크기 설정
    per_device_train_batch_size= 16,    # 학습 시 cpu/gpu에 할당되는 배치의 크기
    per_device_eval_batch_size= 16,     # 평가 시 할당되는 배치 크기
    # 평가 및 저장 주기 설정
    eval_strategy= "epoch",             # 한 epoch마다 평가 수행
    save_strategy= "epoch",             # 한 epoch마다 모델 저장
    # 학습 관련 설정
    num_train_epochs= 2,                # 학습 epoch 수
    learning_rate= 5e-5,                # 옵티마이저의 학습율
    weight_decay= 0.01,                 # 가중치 감소 계수
    warmup_ratio= 0.1,                  # lr의 값을 올리는 비율
    logging_steps= 50,                  # 로그를 출력할 step의 간격
    # 모델 선택 및 저장 기준
    load_best_model_at_end= True,       # 학습이 끝났을 때, 가장 성능이 좋은 모델을 자동 로드
    metric_for_best_model= 'f1',        # 최고의 모델을 판단하는 검증 지표
    greater_is_better= True,            # 평가 지표가 높을 수록 좋은 모델인가?
    # 하드웨어 설정
    fp16= torch.cuda.is_available(),    # (Window) cuda 사용 시 16-bit 혼합정밀도 학습을 할 것인가?
    use_mps_device= ( torch.backends.mps.is_available() if not torch.cuda.is_available() else False ),
                                        # (Mac) M1/M2 등 Apple silicon 가속기 사용 여부
    # 외부 로딩용
    report_to= []
)

In [20]:
# Import Error 발생 시 설치
# !pip install accelerate>=0.26.0

In [21]:
# Trainer -> 모델의 학습을 자동으로 관리하는 Hugging Face의 고수준 API

trainer = Trainer(
    model= model,               # 모델 선택
    args= args,                 # 학습에 사용한 설정값
    train_dataset= train_tok,   # 학습에 사용할 데이터
    eval_dataset= test_tok,     # 평가에 사용할 데이터
    tokenizer= tokenizer,       # 토큰화 함수
    compute_metrics= metrics    # 평가에 사용할 검증 지표 함수
)

C:\Users\student\AppData\Local\Temp\ipykernel_10108\1756556769.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
# 평가 및 예측 테스트
eval_res = trainer.evaluate()
print("평가 결과: ", eval_res)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


평가 결과:  {'eval_loss': 0.7021888494491577, 'eval_model_preparation_time': 0.0064, 'eval_accuracy_score': 0.495, 'eval_f1_score': 0.3129251700680272, 'eval_runtime': 18.4807, 'eval_samples_per_second': 10.822, 'eval_steps_per_second': 0.703}


In [23]:
# 새 문장에 대한 감정 분류
samples = ['정말 감동적인 영화였습니다',
           '지루하고 시간 낭비였습니다']

# 토큰화
enc = tokenizer(
    samples,
    return_tensors= 'pt',   # Pytorch 텐서 형태로 변환
    padding= True,
    truncation= True
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [27]:
with torch.no_grad():
    out = model(**enc)
    probs = torch.softmax(
        out['Logits'], dim = 1
    ).cpu().numpy()

In [36]:
for s, p in zip(samples, probs):
    print(f"{s}: 부정 ({p[0] :.3f}), 긍정 ({p[1] :.3f}) | 예측 ({p.argmax()})")

정말 감동적인 영화였습니다: 부정 (0.507), 긍정 (0.493) | 예측 (0)
지루하고 시간 낭비였습니다: 부정 (0.529), 긍정 (0.471) | 예측 (0)
